In [6]:
from sparql_dataframe import get
from pandas import *

In [13]:
endpoint = "https://query.wikidata.org/sparql"

## Starting query (the one used for the paper)

In [27]:
s_query = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  VALUES ?citizen {wd:Q38 wd:Q172579}
  VALUES ?role {wd:Q6625963 wd:Q49757}
  ?writer wdt:P345 ?imdb ;
          rdfs:label ?writerLabel ;
          (wdt:P106 | p:P106 / ps:P106) ?role ;
          (wdt:P27 | p:P27 / ps:P27) ?citizen.
  FILTER (lang(?writerLabel) = 'en')
}
GROUP BY ?writer ?writerLabel ?imdb 
"""

In [29]:
paper_results = get(endpoint, s_query, True).fillna("")
paper_results

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q193018,Gianni Rodari,nm0734427
1,http://www.wikidata.org/entity/Q3751286,Franco Enna,nm0257850
2,http://www.wikidata.org/entity/Q3767161,Giovanni Drovetti,nm0238144
3,http://www.wikidata.org/entity/Q527124,Vincenzo De Crescenzo,nm1876866
4,http://www.wikidata.org/entity/Q3791865,Idolina Landolfi,nm2285763
...,...,...,...
227,http://www.wikidata.org/entity/Q919103,Ugo Betti,nm0079348
228,http://www.wikidata.org/entity/Q1638370,Giorgio Caproni,nm2516736
229,http://www.wikidata.org/entity/Q587389,Lasse Braun,nm0105808
230,http://www.wikidata.org/entity/Q469656,Paolo Giordano,nm1279468


____________________


## `writers_query`
It retrieves every writer, prosaist, novelist or poet that has the Italian citizenship or lived in a country or historical country whose capital city is now an Italian city (to include also writers living, for example, in the Republic of Florence).
Writers are novelists, poets, prosaists or common "writers". However, when an entity is also a screenwriter, it has to be at least also a novelist or a poet.

Problems:
1. Is it better to restrict the threshold of being at least a poet or a novelist as well as a screenwriter excluding the poet possibility (an author to be included should be a novelist if he/she is a screewriter)?
2. When do we set the threshold of being an "Italian writer"? Is it 1200 good?

In [22]:
writers_query = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  #get writers having an IMDb ID
  ?writer wdt:P345 ?imdb .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
    """

wq_post_1200 = """
SELECT DISTINCT ?writer ?writerLabel ?imdb
WHERE {
  #get writers having an IMDb ID and a Date of Born
  ?writer wdt:P345 ?imdb ;
          wdt:P569 ?dob .
  {
    {
      ?historicalCountry wdt:P706 wd:Q145694 . #historicalCountry is anything located in the Italian peninsula
    } UNION {
      ?historicalCountry wdt:P17 wd:Q38 .     #and anything of the sovereign state of Italy
    }
    ?writer (wdt:P27 | p:P27 / ps:P27) ?historicalCountry . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer (wdt:P27 | p:P27 / ps:P27) ?mcItaly . #get writers having or having had the citizenship of one of those countries
  } UNION {
    VALUES ?mcItaly {wd:Q38 wd:Q172579 wd:Q223936 wd:Q48742118 wd:italy} #modern/contemporary Italy can be one of these entities
    ?writer wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  #However, it seems to provide no addition to the results
    ?capitalCity wdt:P17 ?mcItaly .
  }

  {
    VALUES ?otherRole {wd:Q6625963 wd:Q49757}   #get writers that, if screenwriters, are also either novelists or poets
    ?writer wdt:P106 wd:Q28389 ;
           wdt:P106 ?otherRole.
  }UNION{
    VALUES ?role {wd:Q6625963 wd:Q49757 wd:Q36180 wd:Q12144794}  #get writers that are novelists, poets, prosaists, common wikidata "writers" but NOT screenwriters
    ?writer (wdt:P106 | p:P106 / ps:P106) ?role .
    FILTER NOT EXISTS {?writer wdt:P106 wd:Q28389}
  }
  
  FILTER ("1200-01-01"^^xsd:dateTime <= ?dob) 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

GROUP BY ?writer ?writerLabel ?imdb
"""

In [23]:
writers_df = get(endpoint, writers_query, True).fillna("")
writers_post_1200_df = get(endpoint, wq_post_1200, True).fillna("")

In [24]:
writers_df

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q3645969,Bruno Zanin,nm0953025
1,http://www.wikidata.org/entity/Q558097,Mario Rigoni Stern,nm1167259
2,http://www.wikidata.org/entity/Q1067,Dante Alighieri,nm0019604
3,http://www.wikidata.org/entity/Q3608591,Alberto Spadolini,nm0816553
4,http://www.wikidata.org/entity/Q3615513,Andrea Beltramo,nm1244099
...,...,...,...
732,http://www.wikidata.org/entity/Q23017315,Marco Di Meco,nm8472098
733,http://www.wikidata.org/entity/Q1056872,Cesare Meano,nm0575166
734,http://www.wikidata.org/entity/Q4011253,Victor Cavallo,nm0146872
735,http://www.wikidata.org/entity/Q4015115,Vito Pandolfi,nm0659282


In [25]:
writers_post_1200_df

,writer,writerLabel,imdb
0,http://www.wikidata.org/entity/Q3645969,Bruno Zanin,nm0953025
1,http://www.wikidata.org/entity/Q558097,Mario Rigoni Stern,nm1167259
2,http://www.wikidata.org/entity/Q1067,Dante Alighieri,nm0019604
3,http://www.wikidata.org/entity/Q3608591,Alberto Spadolini,nm0816553
4,http://www.wikidata.org/entity/Q3615513,Andrea Beltramo,nm1244099
...,...,...,...
717,http://www.wikidata.org/entity/Q1042535,Carlo Terron,nm1923000
718,http://www.wikidata.org/entity/Q378674,Arnoldo Foà,nm0289450
719,http://www.wikidata.org/entity/Q993477,Bruno Lauzi,nm0487336
720,http://www.wikidata.org/entity/Q1366593,Paolo Giacometti,nm0316014
